In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.cross_validation import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras import backend

In [4]:
import numpy as np
import os
from image_loader import load

In [5]:
MODEL_NAME = 'coins-regression-0.01-3-1_48_64.h5'

In [6]:
X, y = load('br-coins-regression.zip', ratio=10, task='regression')

250 images to array
500 images to array
750 images to array
1000 images to array
1250 images to array
1500 images to array
1750 images to array
2000 images to array
2250 images to array
2500 images to array
2750 images to array
3000 images to array
3250 images to array
3500 images to array
3750 images to array
4000 images to array
4250 images to array
4500 images to array
4750 images to array
5000 images to array
5250 images to array
5500 images to array
5750 images to array
6000 images to array
Loaded!


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [9]:
def cents_metric(y_true, y_pred):
    return backend.mean(backend.abs(y_true - (5 * backend.round(y_pred / 5))))

In [10]:
if os.path.exists(MODEL_NAME):
    model = load_model(MODEL_NAME, custom_objects={'cents_metric': cents_metric})
    
else:
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=X_train[0].shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
        
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten()) 
    model.add(Dense(64))

    model.add(Activation('linear'))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=[cents_metric])

    model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))
    
print(model.evaluate(X_test, y_test))

603/603 [==============================] - 3s      
[307.62482074085949, 12.885572137721933]


In [ ]:
model.save(MODEL_NAME)

In [11]:
model.evaluate(X, y)

6028/6028 [==============================] - 30s     

[261.69093732827537, 12.10019907100199]